         

# MUSIC APP DATA ANALYSIS AND VISUALIZATION  BY SOMESH KUMAR  

In [36]:
import pandas as pd
import numpy as np
import requests


In [31]:


df_stream=pd.read_json('StreamingHistory0.json')
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']
df_stream.head()



# read your edited Library json file into a pandas dataframe
df_library = pd.read_json('YourLibrary.json')
print(df_library.head())
# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']
# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]
print(df_library.head())






# create final dict as a copy df_stream
df_tableau = df_stream.copy()
# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)
# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])
df_tableau.head()














            artist          album                track  \
0           Halsey     Without Me           Without Me   
1      Alan Walker   Sweet Dreams         Sweet Dreams   
2     Selena Gomez           Rare  Lose You To Love Me   
3      Alan Walker  Alone, Pt. II        Alone, Pt. II   
4  Imagine Dragons        Origins             Bad Liar   

                                    uri  
0  spotify:track:5p7ujcrUXASCNwRaWNHR1C  
1  spotify:track:4uDhts78RdwIDjEqPrYVKp  
2  spotify:track:4l0Mvzj72xxOpRrp6h8nHi  
3  spotify:track:0bMbDctzMmTyK2j74j3nF3  
4  spotify:track:2RSHsoi04658QL5xgQVov3  
            artist          album                track  \
0           Halsey     Without Me           Without Me   
1      Alan Walker   Sweet Dreams         Sweet Dreams   
2     Selena Gomez           Rare  Lose You To Love Me   
3      Alan Walker  Alone, Pt. II        Alone, Pt. II   
4  Imagine Dragons        Origins             Bad Liar   

                                    uri           

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2020-10-10 14:14,Sandip Ssingh,Namo Namo,12390,Sandip Ssingh:Namo Namo,0,NaN,NaN
1,2020-10-10 14:14,Sukhwinder Singh,Saugandh Mujhe Iss Mitti Ki,9416,Sukhwinder Singh:Saugandh Mujhe Iss Mitti Ki,0,NaN,NaN
2,2020-10-10 14:18,Sandip Ssingh,Namo Namo,3653,Sandip Ssingh:Namo Namo,0,NaN,NaN
3,2020-10-10 14:18,Sukhwinder Singh,Saugandh Mujhe Iss Mitti Ki,235333,Sukhwinder Singh:Saugandh Mujhe Iss Mitti Ki,0,NaN,NaN
4,2020-10-10 14:40,Suvarna Tiwari,Ishwar Allah,78476,Suvarna Tiwari:Ishwar Allah,0,NaN,NaN


In [27]:

# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = 'ae5cf17d615645e197a3bf57c057b0cd'
CLIENT_SECRET = '3f062f6b42a54092a6e9ba2119462289'



# generate access token
# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})
# convert the response to JSON
auth_response_data = auth_response.json()
# save the access token
access_token = auth_response_data['access_token']



# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}


# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'


# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}
# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()
# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']
    
    
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)
df_genre.head()



,track_uri,artist_uri,genres
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,"[dance pop, electropop, etherpop, indie poptim..."
1,4uDhts78RdwIDjEqPrYVKp,7vk5e3vY1uw9plTHJAMwjN,[electro house]
2,4l0Mvzj72xxOpRrp6h8nHi,0C8ZW7ezQVs4URX5aX7Kqx,"[dance pop, pop, post-teen pop]"
3,0bMbDctzMmTyK2j74j3nF3,7vk5e3vY1uw9plTHJAMwjN,[electro house]
4,2RSHsoi04658QL5xgQVov3,53XhwfbYqKCa1cC15pYq2q,"[modern rock, pop, rock]"


In [25]:

df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,dance pop
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,electropop
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,etherpop
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,indie poptimism
0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,pop


In [34]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')



In [45]:
a=pd.read_csv('MySpotifyDataTable.csv')
a.head(5)


,Unnamed: 0,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,0,2020-10-10 14:14,Sandip Ssingh,Namo Namo,12390,Sandip Ssingh:Namo Namo,0,NaN,NaN
1,1,2020-10-10 14:14,Sukhwinder Singh,Saugandh Mujhe Iss Mitti Ki,9416,Sukhwinder Singh:Saugandh Mujhe Iss Mitti Ki,0,NaN,NaN
2,2,2020-10-10 14:18,Sandip Ssingh,Namo Namo,3653,Sandip Ssingh:Namo Namo,0,NaN,NaN
3,3,2020-10-10 14:18,Sukhwinder Singh,Saugandh Mujhe Iss Mitti Ki,235333,Sukhwinder Singh:Saugandh Mujhe Iss Mitti Ki,0,NaN,NaN
4,4,2020-10-10 14:40,Suvarna Tiwari,Ishwar Allah,78476,Suvarna Tiwari:Ishwar Allah,0,NaN,NaN


In [44]:
b=pd.read_csv('GenresExpandedTable.csv')
b.head(5)

,Unnamed: 0,track_uri,artist_uri,genres
0,0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,dance pop
1,0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,electropop
2,0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,etherpop
3,0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,indie poptimism
4,0,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,pop


In [ ]:
# TABLEAU LINK >> 

# https://public.tableau.com/views/dappproject/Sheet4?:language=en-US&:display_count=n&:origin=viz_share_link